<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Export-calibration-stream" data-toc-modified-id="Export-calibration-stream-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Export calibration stream</a></span></li><li><span><a href="#Get-the-image-and-the-data" data-toc-modified-id="Get-the-image-and-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get the image and the data</a></span></li><li><span><a href="#Set-the-scale" data-toc-modified-id="Set-the-scale-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set the scale</a></span></li><li><span><a href="#Process-calibration-image" data-toc-modified-id="Process-calibration-image-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Process calibration image</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fit the model</a></span></li><li><span><a href="#Get-the-sigma-calibration-structures-if-necessary" data-toc-modified-id="Get-the-sigma-calibration-structures-if-necessary-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Get the sigma calibration structures if necessary</a></span></li></ul></div>

In [18]:
%matplotlib qt
import f3st
from f3st import deep_reload
deep_reload(f3st)
from f3st import *

import matplotlib.pyplot as plt

### Export calibration stream

In [19]:
calib_strm_savepath = 'calibstrm'

# create safe filename to prevent overwriting
calib_strm_savepath = create_safe_savename(calib_strm_savepath)
export_spot_calibration(calib_strm_savepath, start_time=1, end_time=15, grid=[5, 3])

### Get the image and the data

In [24]:
img_path = 'testing\calib1_001.tif'
data_path = 'testing\calib25-11-19_data.txt'
observing_angle = 45

data = np.loadtxt(data_path, delimiter='\t', skiprows=1)
dwell_times = data[:, 0]

img = read_image(img_path)
plt.imshow(img)

### Set the scale

In [25]:
tracker = select_scale(img)

81, 21
402, 25


In [26]:
scale_bar_size = 2 # in um

scale_bar_px = np.abs(tracker.scale_markers[1] - tracker.scale_markers[0])
ppn = scale_bar_px / scale_bar_size
print('Pixels per nanometer: ', ppn)

Pixels per nanometer:  160.2967741935484


### Process calibration image

In [27]:
# remove the bar and threshold the image
img_nobar = remove_bottom_bar(img)
img_thresh = threshold_image(img_nobar, thresh=None)
# try_all_threshold(img_smooth)
plt.imshow(img_thresh)

In [28]:
# label image regions
label_image, image_label_overlay = get_labelled_image(img_thresh)
labels = filter_small_labels(label_image)
image_label_overlay = label2rgb(label_image, image=img_thresh, bg_label=0)
plt.imshow(image_label_overlay)

In [31]:
# get lengths of the structures
lengths_px = get_lengths_px(label_image)
lengths_perspective = np.sort(lengths_px) / ppn
lengths_true = lengths_perspective / np.sin(np.deg2rad(observing_angle))
dwell_times = dwell_times[-lengths_true.size:]

### Fit the model

In [32]:
# fit the deposit model and get the parameters
deposit_model, popt, _ = DDModel.fit_calibration(dwell_times, lengths_true)

t = np.linspace(0, np.max(dwell_times), 1000)
plt.plot(t, deposit_model(t, *popt), color='tab:orange', linewidth=2)
plt.scatter(dwell_times, lengths_true)
plt.xlabel('t [s]')
plt.ylabel(r'Length [$\mu$m]');

GR:  0.19974639515473672
k:  1.1286493672508695


### Get the sigma calibration structures if necessary

In [165]:
deep_reload(f3st)
from f3st import *
# model = DDModel(gr=popt[0], k=popt[1], sigma=None)
model = RRLModel(gr=popt[0], sigma=None)
sigma_list = [3, 4, 5, 6, 7]
file_path = "sigma_calib"

sigma_strm = get_sigma_structures(model, sigma_list)
sigma_strm.print_time()

sigma_strm.write(create_safe_savename(file_path))

377it [00:00, 586.38it/s]
377it [00:06, 56.06it/s]
377it [00:08, 43.12it/s]
377it [00:06, 59.45it/s]
377it [00:05, 65.18it/s]
377it [00:05, 65.40it/s]


In [169]:
sigma_strm.show_on_screen()